In [0]:
%sql
select * from practice.cat.claims_s limit 3;

In [0]:
#We can't do insert,update,delete in dataframes
#so Delta tables came to picture where we can do  all type of trnasformations

In [0]:
%sql
update practice.cat.claims_s set Street='Mammen zila' where  Street='Mammen Zila'


In [0]:
%sql
SELECT * FROM practice.cat.claims_s where  Street='Mammen zila'



In [0]:
%sql
delete from  practice.cat.claims_s where  Street='Mammen zila'

In [0]:
%sql
insert into practice.cat.claims_s select * from  practice.cat.claims_s where  Street='Mammen zila'

In [0]:
%sql
SELECT * FROM practice.cat.claims_s VERSION AS OF 1 where Street='Mammen zila'

In [0]:
%sql 
DESCRIBE HISTORY practice.cat.claims_s

In [0]:
# Roll back the Delta table to a previous version using RESTORE
spark.sql("""
RESTORE TABLE practice.cat.claims_s TO VERSION AS OF 0
""")

#table restored to version 0 so we can see older version table   

In [0]:

%sql
SELECT * FROM practice.cat.claims_s  where Street='Mammen zila' 

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import *

# Define DeltaTable object
delta_table = DeltaTable.forName(spark, "practice.cat.claims_s")

# Update: Set Street='Mammen zila' where Street='Mammen Zila'
delta_table.update(
    condition=expr("Street = 'Mammen Zila'"),
    set={"Street": expr("'Mammen zila'")}
)

# Delete: Remove rows where Street='Mammen zila'
delta_table.delete(condition=expr("Street = 'Mammen zila'"))

# Insert: Insert rows (example, duplicating rows where Street='Mammen zila')
df_to_insert = delta_table.toDF().filter("Street = 'Mammen zila'")
df_to_insert.write.format("delta").mode("append").saveAsTable("practice.cat.claims_s")

# Time travel: Read table as of version 1
df_time_travel = spark.read.format("delta").option("versionAsOf", 1).table("practice.cat.claims_s")
display(df_time_travel)

In [0]:
history_df = delta_table.history()
display(history_df)